In [ ]:
import pandas as pd

# df_train = pd.read_csv("vuln_data_train.csv")
# df_train

df_train = pd.read_csv("vuln_data_val.csv")
df_train

,file_name,vuln_title,vuln_explanation,severity,vuln_recommendation,vuln_code
0,123.md,`AuraBalRewardPool` charges a penalty to all u...,"When claiming their rewards, users are charged...",medium,Don't charge the penalty if the locker has bee...,"function _lock(address _account, uint256 _amou..."
1,106.md,Might not get desired min loan amount if `_ori...,Admins can update the origination fee by calli...,medium,"Reconsider how the min loan amount works. Imo,...",function createLoan(\n uint256 collater...
2,41.md,Basket becomes unusable if everybody burns the...,"While handling the fees, the contract calculat...",medium,Add a check to `handleFees: if totalSupply= 0`...,function handleFees() private {\n if (l...
3,5.md,Out-of-bound index access in function `getAnch...,Out-of-bound index access is possible in the f...,low,Consider using `arrayPrices.length/2` as the i...,function getAnchorPrice() public view returns ...
4,70.md,Oracle returns an improperly scaled USDV/VADER...,The LBT oracle does not properly scale values ...,high,Go over oracle calculation again to ensure tha...,function _calculateUSDVPrice(\n uint256...
...,...,...,...,...,...,...
137,114.md,Owner or Managers can rug Aave rewards,A malicious owner or manager can steal all Aav...,medium,Use a poolAddressesProviderRegistry-like contr...,function claimRewards(address _to) external on...
138,3.md,Different solidity version in UniswapStyleLib.sol,The solidity version in UniswapStyleLib.sol (>...,low,It is cleaner to have the same solidity versio...,function getAmountOut(\n uint256 amount...
139,52.md,Incorrect Accrual Of `sumNative` and `sumUSD` ...,The `TwapOracle.consult()` function iterates o...,high,To calculate the correct consultation of a giv...,function consult(address token) public view re...
140,35.md,Wrong usage of `positionId` in `ConcentratedLi...,In the `subscribe` function of `ConcentratedLi...,high,Change `positionId` to `incentiveId` in the re...,"function subscribe(uint256 positionId, uint256..."


In [ ]:
import re
from unsloth import FastLanguageModel
from unsloth.chat_templates import get_chat_template
from vllm import SamplingParams

def extract_answer(text: str) -> str | None:
    answer = text.split("</think>\n\n")[-1]
    if "<think>" in answer:
        return None
    return answer.strip()

# Sampling parameters
sampling_params = SamplingParams(
    temperature=0.7,
    top_k=20,
    top_p=0.8,
    min_p=0,
    max_tokens=8192,
)

THINKER_SYSTEM_PROMPT = [
    """You are an LLM Chain-of-Thought Generator. That answer straight the Chain-of-Thought without any commentary or additional information.""",
    """You are a code analyst specialized in security. Your job is to provide a detailed Chain-of-Thought analysis without adding explanations or commentary outside the logical reasoning steps.""",
    """You are a reviewer performing a recommendation patch of vulnerability walk-through. Just present your thought process step-by-step without external opinions or extra commentary.""",
    """You are a vulnerability patcher. Present only your internal reasoning steps that logically lead from the code to the known issue. No narration or conclusions outside of your investigation.""",
    """You're a mentor walking a junior auditor through a known mitigation patch. Only share your logical process without any fluff, summaries, or external commentary."""
]

THINKER_USER_PROMPT = [
    """Given the vulnerable solidity code, vulnerability explanation and the ground truth mitigation in the context.
Your task is to give Chain-of-Thought how we can give mitigation of the vulnerability just from the code and explanation.
Your Chain-of-Thought should has: the understanding the use case of the code, analyzing the vulnerabilities and give the final recommendation answer that is the ground truth mitigation approach.
Give your answer as a human thinking, so start with something like: "Okay so we have ...", "Okay lets see...", etc.

**Vulnerable Code:**
```solidity
{code}```

**Vulnerability Explanation:**  
{vuln_explanation}

**Recommendation:**  
{ground_truth}
""",
    
    """You are given a vulnerable Solidity smart contract, the known vulnerability and the ground truth recommendation steps.  
Your task is to think through how a security engineer would identify that recommendation just by looking at the code and the vulnerability explanation.  

Structure your reasoning like natural human thinking, with transitions such as:  
"Alright, the first thing I notice...", "If I follow the logic here...", etc. 
And start with something like:
"Okay so we have ...", "Okay lets see...", etc. 

**Code:**  
```solidity
{code}```

**Vulnerability Explanation:**  
{vuln_explanation}

**Recommendation:**  
{ground_truth}
""",
    
    """You're reviewing this smart contract with the aim of discovering the recommendation of the vulnerability that exist in the code.

Explain how you'd find the recommendation/mitigation by thinking through the code and vulnerability explanation.  
Keep the tone human and reflective, as if explaining to a colleague. Start with lines like:  
"Okay, I'm looking at this function...", "Okay lets check the function....At first glance, this looks fine, but...", etc. 

**Smart Contract Code:**  
```solidity
{code}```

**Vulnerability:**  
{vuln_explanation}

**Recommendation:**  
{ground_truth}
""",
    
    """Imagine you're investigating this Solidity contract like a security patcher.

Work your way toward the recommendation to patch the vulnerability step-by-step. Let your Chain-of-Thought feel like natural human deduction. Use transitions like:  
"Hmm, this function seems suspicious...", "Wait a second, if this variable isn’t protected...", etc.  
And start with something like "Okay lets check the functionality...", "Okay lets take a look the function", etc. 

**Vulnerable Code:**  
```solidity
{code}```

**Known Vulnerability:**  
{vuln_explanation}

**Recommendation:**  
{ground_truth}
""",
    
    """You are mentoring someone in finding recommendation of the vulnerability. You already know what the recommendation is, but you're walking them through how you'd discover it naturally just from reading the Solidity code and the vulnerability explanation.

Make your Chain-of-Thought sound human and intuitive
"Okay let’s take a look at how this function behaves...", "Okay lets notice how this variable is exposed...", etc. 

**Review Code:**  
```solidity
{code}```

**Known Vulnerability:**  
{vuln_explanation}

**Recommendation:**  
{ground_truth}
"""
]

thinker_model_name = "unsloth/Qwen3-14B-unsloth-bnb-4bit"
thinker_max_seq_length = 8192  # Adjust based on your requirements
thinker_load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

thinker_model, thinker_tokenizer = FastLanguageModel.from_pretrained(
    model_name=thinker_model_name,
    load_in_4bit = thinker_load_in_4bit,
    fast_inference=True,
    max_seq_length=thinker_max_seq_length,
)
FastLanguageModel.for_inference(thinker_model) 


def llm_thinker(idx, code, vuln_exp, ground_truth):
    messages = [
        {"role": "system", "content": THINKER_SYSTEM_PROMPT[idx]},
        {"role": "system", "content": THINKER_USER_PROMPT[idx].format(code=code, vuln_explanation=vuln_exp, ground_truth=ground_truth)},
    ]
    
    # Apply chat template
    text = thinker_tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        tokenize=False,
        enable_thinking=False,
    )
    
    try:
        # Generate output
        output = thinker_model.fast_generate(
            [text],
            sampling_params=sampling_params,
        )[0].outputs[0].text
    except Exception as e:
        print(f"Error generating {idx}: {e}")
        return None
    
    return extract_answer(output)

/usr/local/lib/python3.10/dist-packages/torch/utils/_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth Zoo will now patch everything to make training faster!


/usr/local/lib/python3.10/dist-packages/torch/utils/_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


...

INFO 06-12 02:10:47 [punica_selector.py:18] Using PunicaWrapperGPU.


INFO 06-12 02:10:48 [model_runner.py:1140] Model loading took 10.8959 GiB and 10.128788 seconds
INFO 06-12 02:10:55 [worker.py:287] Memory profiling takes 6.58 seconds
INFO 06-12 02:10:55 [worker.py:287] the current vLLM instance can use total_gpu_memory (31.75GiB) x gpu_memory_utilization (0.49) = 15.70GiB
INFO 06-12 02:10:55 [worker.py:287] model weights take 10.90GiB; non_torch_memory takes 0.07GiB; PyTorch activation peak memory takes 1.36GiB; the rest of the memory reserved for KV Cache is 3.38GiB.
INFO 06-12 02:10:55 [executor_base.py:112] # cuda blocks: 1382, # CPU blocks: 2457
INFO 06-12 02:10:55 [executor_base.py:117] Maximum concurrency for 8192 tokens per request: 2.70x
INFO 06-12 02:11:00 [model_runner.py:1450] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider decr

Capturing CUDA graph shapes: 100%|█████████████████████████████████████████████████████████| 27/27 [00:36<00:00,  1.35s/it]

INFO 06-12 02:11:36 [model_runner.py:1592] Graph capturing finished in 37 secs, took 0.94 GiB
INFO 06-12 02:11:36 [llm_engine.py:437] init engine (profile, create kv cache, warmup model) took 48.50 seconds


Unsloth: Just some info: will skip parsing ['post_feedforward_layernorm', 'pre_feedforward_layernorm']
Unsloth: Just some info: will skip parsing ['post_feedforward_layernorm', 'pre_feedforward_layernorm']


In [ ]:
import os
import pandas as pd
import random
from tqdm import tqdm

# output_path = 'vuln_data_train_with_recom_CoT.csv'
output_path = 'vuln_data_val_with_recom_CoT.csv'

# Load existing processed rows if the file already exists
if os.path.exists(output_path):
    df_existing = pd.read_csv(output_path)
    processed_indices = set(df_existing.index)
else:
    df_existing = pd.DataFrame()
    processed_indices = set()

# Prepare 5 columns in the original df if not already present
for i in range(5):
    if f'CoT_{i}' not in df_train.columns:
        df_train[f'CoT_{i}'] = None

# Process and append to CSV one by one
for idx in tqdm(range(len(df_train))):
    if idx in processed_indices:
        print(f"Skipped Ids: {idx}")
        continue  # Skip already processed rows

    answers = []
    for i in range(5):
        answer = llm_thinker(i, df_train.loc[idx, 'vuln_code'], df_train.loc[idx, 'vuln_explanation'], df_train.loc[idx, 'vuln_recommendation']) 
        answers.append(answer)

    random.shuffle(answers)

    for i, ans in enumerate(answers):
        df_train.at[idx, f'CoT_{i}'] = ans

    # Extract just the current row and append to file
    row_df = df_train.loc[[idx]]  # Double brackets to keep it as a DataFrame
    row_df.to_csv(output_path, mode='a', header=not os.path.exists(output_path), index=True)

Processed prompts: 100%|██████████████| 1/1 [00:07<00:00,  7.43s/it, est. speed input: 127.99 toks/s, output: 27.99 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:12<00:00, 12.98s/it, est. speed input: 73.53 toks/s, output: 29.21 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:15<00:00, 15.27s/it, est. speed input: 61.65 toks/s, output: 29.42 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:15<00:00, 15.25s/it, est. speed input: 62.04 toks/s, output: 29.38 toks/s]

Processed prompts: 100%|██████████████| 1/1 [00:12<00:00, 12.08s/it, est. speed input: 144.53 toks/s, output: 28.08 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:19<00:00, 19.11s/it, est. speed input: 91.48 toks/s, output: 29.10 toks/s]

Processed prompts: 100%|██████████████| 1/1 [00:13<00:00, 13.10s/it, est. speed input: 132.51 toks/s, output: 28.34 toks/s]

Processed prompts: 100%|██████████████| 1/1 [00:12<00:00, 12.44s/it, est. speed input: 139.94 toks/s, output: 28.07 toks/s]



Processed prompts: 100%|██████████████| 1/1 [00:13<00:00, 13.61s/it, est. speed input: 216.69 toks/s, output: 26.76 toks/s]

Processed prompts: 100%|██████████████| 1/1 [00:18<00:00, 18.77s/it, est. speed input: 157.23 toks/s, output: 27.70 toks/s]

Processed prompts: 100%|██████████████| 1/1 [00:15<00:00, 15.42s/it, est. speed input: 190.52 toks/s, output: 27.24 toks/s]

Processed prompts: 100%|██████████████| 1/1 [00:19<00:00, 19.31s/it, est. speed input: 152.41 toks/s, output: 28.07 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:11<00:00, 11.56s/it, est. speed input: 70.07 toks/s, output: 29.76 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:10<00:00, 10.85s/it, est. speed input: 74.92 toks/s, output: 29.67 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:15<00:00, 15.85s/it, est. speed input: 50.48 toks/s, output: 30.16 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:11<00:00, 11.37s/it, est. speed input: 70.83 toks/s, output: 29.74 toks/s]



Processed prompts: 100%|██████████████| 1/1 [00:14<00:00, 14.51s/it, est. speed input: 106.28 toks/s, output: 29.09 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:16<00:00, 16.22s/it, est. speed input: 42.66 toks/s, output: 30.27 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:15<00:00, 15.53s/it, est. speed input: 44.76 toks/s, output: 30.21 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:19<00:00, 19.79s/it, est. speed input: 34.46 toks/s, output: 30.42 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:15<00:00, 15.55s/it, est. speed input: 44.18 toks/s, output: 30.22 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:06<00:00,  6.30s/it, est. speed input: 67.47 toks/s, output: 29.53 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:12<00:00, 12.23s/it, est. speed input: 35.01 toks/s, output: 30.34 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:15<00:00, 15.25s/it, est. speed input: 27.21 toks/s, output: 30.49 toks/s]



Processed prompts: 100%|███████████████| 1/1 [00:15<00:00, 15.91s/it, est. speed input: 17.61 toks/s, output: 30.69 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:08<00:00,  8.21s/it, est. speed input: 34.72 toks/s, output: 30.22 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:14<00:00, 14.91s/it, est. speed input: 59.08 toks/s, output: 30.05 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:13<00:00, 13.53s/it, est. speed input: 65.36 toks/s, output: 29.87 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:19<00:00, 19.26s/it, est. speed input: 45.23 toks/s, output: 30.27 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:18<00:00, 18.48s/it, est. speed input: 47.41 toks/s, output: 30.25 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:11<00:00, 11.45s/it, est. speed input: 87.16 toks/s, output: 29.52 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:14<00:00, 14.84s/it, est. speed input: 67.46 toks/s, output: 29.86 toks/s]



Processed prompts: 100%|███████████████| 1/1 [00:12<00:00, 12.60s/it, est. speed input: 76.19 toks/s, output: 29.68 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:19<00:00, 19.07s/it, est. speed input: 49.67 toks/s, output: 30.16 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:13<00:00, 13.67s/it, est. speed input: 69.65 toks/s, output: 29.78 toks/s]

Processed prompts: 100%|██████████████| 1/1 [00:07<00:00,  7.95s/it, est. speed input: 163.75 toks/s, output: 28.07 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:15<00:00, 15.09s/it, est. speed input: 86.42 toks/s, output: 29.49 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:23<00:00, 23.63s/it, est. speed input: 54.64 toks/s, output: 30.01 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:15<00:00, 15.33s/it, est. speed input: 84.58 toks/s, output: 29.50 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:10<00:00, 10.63s/it, est. speed input: 64.57 toks/s, output: 29.84 toks/s]



...

Processed prompts: 100%|███████████████| 1/1 [00:09<00:00,  9.75s/it, est. speed input: 47.51 toks/s, output: 30.07 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:12<00:00, 12.74s/it, est. speed input: 35.32 toks/s, output: 30.30 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:07<00:00,  7.28s/it, est. speed input: 62.51 toks/s, output: 29.67 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:11<00:00, 11.68s/it, est. speed input: 79.83 toks/s, output: 29.55 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:14<00:00, 14.64s/it, est. speed input: 63.89 toks/s, output: 29.86 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:20<00:00, 20.10s/it, est. speed input: 45.88 toks/s, output: 30.16 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:15<00:00, 15.32s/it, est. speed input: 60.52 toks/s, output: 29.90 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:06<00:00,  6.43s/it, est. speed input: 54.47 toks/s, output: 29.73 toks/s]



Processed prompts: 100%|███████████████| 1/1 [00:17<00:00, 17.16s/it, est. speed input: 46.04 toks/s, output: 30.19 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:19<00:00, 19.50s/it, est. speed input: 40.67 toks/s, output: 30.26 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:19<00:00, 19.57s/it, est. speed input: 39.87 toks/s, output: 30.31 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:16<00:00, 16.26s/it, est. speed input: 48.29 toks/s, output: 30.14 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:12<00:00, 12.22s/it, est. speed input: 73.61 toks/s, output: 29.64 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:12<00:00, 12.41s/it, est. speed input: 72.69 toks/s, output: 29.65 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:15<00:00, 15.65s/it, est. speed input: 56.83 toks/s, output: 30.05 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:14<00:00, 14.36s/it, est. speed input: 62.29 toks/s, output: 29.96 toks/s]



Processed prompts: 100%|███████████████| 1/1 [00:08<00:00,  8.94s/it, est. speed input: 39.71 toks/s, output: 29.98 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:11<00:00, 11.33s/it, est. speed input: 31.62 toks/s, output: 30.29 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:12<00:00, 12.85s/it, est. speed input: 26.85 toks/s, output: 30.43 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:10<00:00, 10.57s/it, est. speed input: 33.13 toks/s, output: 30.29 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:11<00:00, 11.45s/it, est. speed input: 31.36 toks/s, output: 30.23 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:13<00:00, 13.24s/it, est. speed input: 27.35 toks/s, output: 30.30 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:14<00:00, 14.17s/it, est. speed input: 24.64 toks/s, output: 30.36 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:09<00:00,  9.88s/it, est. speed input: 35.84 toks/s, output: 30.07 toks/s]



Processed prompts: 100%|███████████████| 1/1 [00:14<00:00, 14.04s/it, est. speed input: 67.54 toks/s, output: 29.85 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:13<00:00, 13.27s/it, est. speed input: 95.83 toks/s, output: 29.40 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:15<00:00, 15.08s/it, est. speed input: 84.51 toks/s, output: 29.58 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:19<00:00, 19.36s/it, est. speed input: 65.15 toks/s, output: 29.91 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:15<00:00, 15.14s/it, est. speed input: 83.63 toks/s, output: 29.60 toks/s]

Processed prompts: 100%|██████████████| 1/1 [00:11<00:00, 11.51s/it, est. speed input: 123.97 toks/s, output: 28.76 toks/s]

Processed prompts: 100%|██████████████| 1/1 [00:12<00:00, 12.84s/it, est. speed input: 111.36 toks/s, output: 28.97 toks/s]

Processed prompts: 100%|██████████████| 1/1 [00:14<00:00, 14.03s/it, est. speed input: 101.04 toks/s, output: 29.16 toks/s]



Processed prompts: 100%|███████████████| 1/1 [00:13<00:00, 13.35s/it, est. speed input: 27.87 toks/s, output: 30.49 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:11<00:00, 11.87s/it, est. speed input: 31.77 toks/s, output: 30.42 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:09<00:00,  9.32s/it, est. speed input: 74.06 toks/s, output: 29.62 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:16<00:00, 16.13s/it, est. speed input: 42.97 toks/s, output: 30.26 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:18<00:00, 18.99s/it, est. speed input: 35.82 toks/s, output: 30.39 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:16<00:00, 16.32s/it, est. speed input: 41.99 toks/s, output: 30.28 toks/s]

Processed prompts: 100%|██████████████| 1/1 [00:10<00:00, 10.52s/it, est. speed input: 136.07 toks/s, output: 28.53 toks/s]

Processed prompts: 100%|███████████████| 1/1 [00:16<00:00, 16.78s/it, est. speed input: 85.47 toks/s, output: 29.44 toks/s]

